In [41]:
!pip install jupyterthemes
!jt -t monokai -f fira -fs 13 -nf Hack -nfs 12 -N -kl -cursw 5 -cursc r -cellw 95% -T

  Using cached https://files.pythonhosted.org/packages/8a/08/9dee6dfd7f2aad6c30282d55c8f495b4dc1e4747b4e2bdbeb80572ddf312/jupyterthemes-0.20.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f8/d2/665cda6614e3556eaeb7553a3a2963624c2e3bc9636777a1bb654b87b027/lesscpy-0.14.0-py2.py3-none-any.whl



	One of the fonts you requested is not available
	Setting all fonts to default


Traceback (most recent call last):
  File "c:\users\sxc0126\appdata\local\continuum\anaconda3\envs\project_c\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\sxc0126\appdata\local\continuum\anaconda3\envs\project_c\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\sxc0126\AppData\Local\Continuum\anaconda3\envs\project_c\Scripts\jt.exe\__main__.py", line 7, in <module>
  File "c:\users\sxc0126\appdata\local\continuum\anaconda3\envs\project_c\lib\site-packages\jupyterthemes\__init__.py", line 314, in main
    dfonts=args.defaultfonts)
  File "c:\users\sxc0126\appdata\local\continuum\anaconda3\envs\project_c\lib\site-packages\jupyterthemes\__init__.py", line 80, in install_theme
    dfonts=dfonts)
  File "c:\users\sxc0126\appdata\local\continuum\anaconda3\envs\project_c\lib\site-packages\jupyterthemes\stylefx.py", line 156, in set_font_properties
    nbfont, nbfontpath = stored_font_dicts(nbfont)
ValueError: not eno

##  Featuretools
is an open-source Python package for automatically creating new features from multiple tables of structured, related data. It is ideal tool for problems such as the Home Credit Default Risk competition where there are several related tables that need to be combined into a single dataframe for training (and one for testing).

## Feature Engineering
The objective of feature engineering is to create new features (alos called explantory variables or predictors) to represent as much information from an entire dataset in one table. Typically, this process is done by hand using pandas operations such as groupby, agg, or merge and can be very tedious. Moreover, manual feature engineering is limited both by human time constraints and imagination: we simply cannot conceive of every possible feature that will be useful. 

### Automated feature engineering
aims to help the data scientist with the problem of feature creation by automatically building hundreds or thousands of new features from a dataset. Featuretools - the only library for automated feature engineering at the moment - will not replace the data scientist, but it will allow her to focus on more valuable parts of the machine learning pipeline, such as delivering robust models into production.

In [1]:
!pip install featuretools

  Created wheel for toolz: filename=toolz-0.10.0-cp37-none-any.whl size=55581 sha256=0858cce96aaca80826621cadca55be3e7c975efb3e1c2d00cba31af3cd76449f
  Stored in directory: C:\Users\sxc0126\AppData\Local\pip\Cache\wheels\e1\8b\65\3294e5b727440250bda09e8c0153b7ba19d328f661605cb151
  Created wheel for locket: filename=locket-0.2.0-cp37-none-any.whl size=4045 sha256=78079e51f53e07061149c7a4a71c377ce9c2e39dbf021b3d43b789cf41623063
  Stored in directory: C:\Users\sxc0126\AppData\Local\pip\Cache\wheels\26\1e\e8\4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
Successfully built toolz locket


In [2]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# matplotlit and seaborn for visualizations
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 22
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

#### Problem
The Home Credit Default Risk competition is a supervised classification machine learning task. The objective is to use historical financial and socioeconomic data to predict whether or not an applicant will be able to repay a loan. This is a standard supervised classification task:

- Supervised: The labels are included in the training data and the goal is to train a model to learn to predict the labels from the features
- Classification: The label is a binary variable, 0 (will repay loan on time), 1 (will have difficulty repaying loan)

#### Read in Data and Create Small Datasets
We will read in the full dataset, sort by the SK_ID_CURR and keep only the first 1000 rows to make the calculations feasible. Later we can convert to a script and run with the entire datasets.

In [5]:
# File system manangement
import os
# List files available
print(os.listdir("../input/home-credit-default-risk"))

['application_test.csv', 'application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv', 'sample_submission.csv']


In [6]:
# Read in the datasets and limit to the first 1000 rows (sorted by SK_ID_CURR) 
# This allows us to actually see the results in a reasonable amount of time! 
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index(drop = True).loc[:1000, :]
bureau_balance = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv').sort_values('SK_ID_BUREAU').reset_index(drop = True).loc[:1000, :]
cash = pd.read_csv('../input/home-credit-default-risk/POS_CASH_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
credit = pd.read_csv('../input/home-credit-default-risk/credit_card_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
previous = pd.read_csv('../input/home-credit-default-risk/previous_application.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
installments = pd.read_csv('../input/home-credit-default-risk/installments_payments.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]

We'll join the train and test set together but add a separate column identifying the set. This is important because we are going to want to apply the same exact procedures to each dataset. It's safest to just join them together and treat them as a single dataframe.

(I'm not sure if this is allowing data leakage into the train set and if these feature creation operations should be applied separately. Any thoughts would be much appreciated!)



In [18]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 24 columns):
credit_index                  1001 non-null int32
SK_ID_PREV                    1001 non-null int64
SK_ID_CURR                    1001 non-null int64
MONTHS_BALANCE                1001 non-null int64
AMT_BALANCE                   1001 non-null float64
AMT_CREDIT_LIMIT_ACTUAL       1001 non-null int64
AMT_DRAWINGS_ATM_CURRENT      805 non-null float64
AMT_DRAWINGS_CURRENT          1001 non-null float64
AMT_DRAWINGS_OTHER_CURRENT    805 non-null float64
AMT_DRAWINGS_POS_CURRENT      805 non-null float64
AMT_INST_MIN_REGULARITY       948 non-null float64
AMT_PAYMENT_CURRENT           813 non-null float64
AMT_PAYMENT_TOTAL_CURRENT     1001 non-null float64
AMT_RECEIVABLE_PRINCIPAL      1001 non-null float64
AMT_RECIVABLE                 1001 non-null float64
AMT_TOTAL_RECEIVABLE          1001 non-null float64
CNT_DRAWINGS_ATM_CURRENT      805 non-null float64
CNT_DRAWINGS_CURRENT     

In [7]:
# Add identifying column
# 트레인과 테스트를 하나로 합친다. 합치면서 테스트에는 np.nan을 타겟에 넣어준다.
app_train['set'] = 'train'
app_test['set'] = 'test'
app_test["TARGET"] = np.nan

# Append the dataframes
app = app_train.append(app_test, ignore_index = True)

## Featuretools Basics
Featuretools is an open-source Python library for automatically creating features out of a set of related tables using a technique called deep feature synthesis. Automated feature engineering, like many topics in machine learning, is a complex subject built upon a foundation of simpler ideas. By going through these ideas one at a time, we can build up our understanding of how featuretools which will later allow for us to get the most out of it.

There are a few concepts that we will cover along the way:

- Entities and EntitySets
- Relationships between tables
- Feature primitives: aggregations and transformations
- Deep feature synthesis

#### Entities and Entitysets
**An entity** is simply a table or in Pandas, **a dataframe**. 
- The observations are in the rows and the features in the columns. 
- An entity in featuretools must have a unique index where none of the elements are duplicated. Currently, only app, bureau, and previous have unique indices (```SK_ID_CURR, SK_ID_BUREAU, and SK_ID_PREV``` respectively). For the other dataframes, we **must pass in make_index = True** and then specify the name of the index. Entities can also have time indices where each entry is identified by a unique time. (There are not datetimes in any of the data, but there are relative times, given in months or days, that we could consider treating as time variables).
- 프라이머리 키가 꼭 모든 테이블에 있어야 한다는 것. 없는 테이블이 있으면 make_index = True를 꼭 해줄것.

**An EntitySet** is a collection of tables and the relationships between them. This can be thought of a data structute with its own methods and attributes. Using an EntitySet allows us to group together multiple tables and manipulate them much quicker than individual tables.

First we'll make an empty entityset named clients to keep track of all the data.


In [9]:
# Entity set with id applications
# 엔티티세트 클라이언츠라는 엔티티세트 생성
es = ft.EntitySet(id = 'clients')

Now we define each entity, or table of data. We need to pass in an index if the data has one or make_index = True if not. Featuretools will automatically infer the types of variables, but we can also change them if needed. For intstance, if we have a categorical variable that is represented as an integer we might want to let featuretools know the right type.
- 카테고리 변수가 정수로 되어 있을 경우도 해결할 수 있도록 해보자.

In [10]:
# Entities with a unique index
es = es.entity_from_dataframe(entity_id = 'app', dataframe = app, index = 'SK_ID_CURR')

es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau, index = 'SK_ID_BUREAU')

es = es.entity_from_dataframe(entity_id = 'previous', dataframe = previous, index = 'SK_ID_PREV')

# Entities that do not have a unique index
es = es.entity_from_dataframe(entity_id = 'bureau_balance', dataframe = bureau_balance, 
                              make_index = True, index = 'bureaubalance_index')

es = es.entity_from_dataframe(entity_id = 'cash', dataframe = cash, 
                              make_index = True, index = 'cash_index')

es = es.entity_from_dataframe(entity_id = 'installments', dataframe = installments,
                              make_index = True, index = 'installments_index')

es = es.entity_from_dataframe(entity_id = 'credit', dataframe = credit,
                              make_index = True, index = 'credit_index')

### Relationships
Relationships are a fundamental concept not only in featuretools, but in any relational database. The best way to think of a one-to-many relationship is with the analogy of parent-to-child. A parent is a single individual, but can have mutliple children. The children can then have multiple children of their own. In a parent table, each individual has a single row. Each individual in the parent table can have multiple rows in the child table.

>As an example, the app dataframe has one row for each client ```(SK_ID_CURR)``` while the bureau dataframe has multiple previous loans ```(SK_ID_PREV)``` for each parent ```(SK_ID_CURR)```. Therefore, the bureau dataframe is the child of the app dataframe. The bureau dataframe in turn is the parent of bureau_balance because each loan has one row in bureau but multiple monthly records in bureau_balance.

In [11]:
es

Entityset: clients
  Entities:
    app [Rows: 2002, Columns: 123]
    bureau [Rows: 1001, Columns: 17]
    previous [Rows: 1001, Columns: 37]
    bureau_balance [Rows: 1001, Columns: 4]
    cash [Rows: 1001, Columns: 9]
    installments [Rows: 1001, Columns: 9]
    credit [Rows: 1001, Columns: 24]
  Relationships:
    No relationships

In [12]:
print('Parent: app, Parent Variable: SK_ID_CURR\n\n', app.iloc[:, 111:115].head())
print('\nChild: bureau, Child Variable: SK_ID_CURR\n\n', bureau.iloc[10:30, :4].head())

Parent: app, Parent Variable: SK_ID_CURR

    SK_ID_CURR  TARGET  TOTALAREA_MODE WALLSMATERIAL_MODE
0      100002     1.0          0.0149       Stone, brick
1      100003     0.0          0.0714              Block
2      100004     0.0             NaN                NaN
3      100006     0.0             NaN                NaN
4      100007     0.0             NaN                NaN

Child: bureau, Child Variable: SK_ID_CURR

     SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY
10      100002       6158905        Closed      currency 1
11      100002       6158906        Closed      currency 1
12      100002       6158907        Closed      currency 1
13      100002       6158908        Closed      currency 1
14      100002       6158909        Active      currency 1


The ```SK_ID_CURR``` "100002" has one row in the parent table and multiple rows in the child.

Two tables are linked via a shared variable. The ```app``` and ```bureau``` dataframe are linked by the ```SK_ID_CURR``` variable while the bureau and bureau_balance dataframes are linked with the SK_ID_BUREAU. 
- Altogether, there are a total of 6 relationships between the tables. Below we specify all six relationships and then add them to the EntitySet.

In [14]:
# Relationship between app and bureau
r_app_bureau = ft.Relationship(es['app']['SK_ID_CURR'], es['bureau']['SK_ID_CURR']) #ft: feature tool

# Relationship between bureau and bureau balance
r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])

# Relationship between current app and previous apps
r_app_previous = ft.Relationship(es['app']['SK_ID_CURR'], es['previous']['SK_ID_CURR'])

# Relationships between previous apps and cash, installments, and credit
r_previous_cash = ft.Relationship(es['previous']['SK_ID_PREV'], es['cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['previous']['SK_ID_PREV'], es['installments']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['previous']['SK_ID_PREV'], es['credit']['SK_ID_PREV'])

In [15]:
# Add in the defined relationships
es = es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,
                           r_previous_cash, r_previous_installments, r_previous_credit])
# Print out the EntitySet
es

Entityset: clients
  Entities:
    app [Rows: 2002, Columns: 123]
    bureau [Rows: 1001, Columns: 17]
    previous [Rows: 1001, Columns: 37]
    bureau_balance [Rows: 1001, Columns: 4]
    cash [Rows: 1001, Columns: 9]
    installments [Rows: 1001, Columns: 9]
    credit [Rows: 1001, Columns: 24]
  Relationships:
    bureau.SK_ID_CURR -> app.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous.SK_ID_CURR -> app.SK_ID_CURR
    cash.SK_ID_PREV -> previous.SK_ID_PREV
    installments.SK_ID_PREV -> previous.SK_ID_PREV
    credit.SK_ID_PREV -> previous.SK_ID_PREV

Slightly advanced note: we need to be careful to not create a diamond graph where there are multiple paths from a parent to a child. 
>If we directly link app and cash via SK_ID_CURR; previous and cash via SK_ID_PREV; and app and previous via SK_ID_CURR, then we have created two paths from app to cash. This results in ambiguity, 

so the approach we have to take instead is to link app to cash through previous. 

We establish a relationship between previous (the parent) and cash (the child) using SK_ID_PREV. Then we establish a relationship between app (the parent) and previous (now the child) using SK_ID_CURR. Then featuretools will be able to create features on app derived from both previous and cash by stacking multiple primitives.

All entities in the entity can be related to each other. In theory this allows us to calculate features for any of the entities, but in practice, we will only calculate features for the app dataframe since that will be used for training/testing.

## Feature Primitives
A feature primitive is an operation applied to a table or a set of tables to create a feature. These represent simple calculations, many of which we already use in manual feature engineering, that can be stacked on top of each other to create complex features. Feature primitives fall into two categories:

- Aggregation: function that groups together child datapoints for each parent and then calculates a statistic such as mean, min, max, or standard deviation. An example is calculating the maximum previous loan amount for each client. An aggregation works across multiple tables using relationships between tables.

- Transformation: an operation applied to one or more columns in a single table. An example would be taking the absolute value of a column, or finding the difference between two columns in one table.
A list of the available features primitives in featuretools can be viewed below.

In [22]:
# List the primitives in a dataframe
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(30)

,name,type,description
0,n_most_common,aggregation,Determines the `n` most common elements.
1,time_since_first,aggregation,Calculates the time elapsed since the first datetime (in seconds).
2,num_true,aggregation,Counts the number of `True` values.
3,time_since_last,aggregation,Calculates the time elapsed since the last datetime (default in seconds).
4,any,aggregation,Determines if any value is 'True' in a list.
5,first,aggregation,Determines the first value in a list.
6,mode,aggregation,Determines the most commonly repeated value.
7,sum,aggregation,"Calculates the total addition, ignoring `NaN`."
8,std,aggregation,"Computes the dispersion relative to the mean value, ignoring `NaN`."
9,trend,aggregation,Calculates the trend of a variable over time.


In [20]:
primitives[primitives['type'] == 'transform'].head(10)

,name,type,description
22,divide_numeric_scalar,transform,Divide each element in the list by a scalar.
23,multiply_numeric_scalar,transform,Multiply each element in the list by a scalar.
24,less_than,transform,Determines if values in one list are less than another list.
25,num_characters,transform,Calculates the number of characters in a string.
26,week,transform,Determines the week of the year from a datetime.
27,or,transform,Element-wise logical OR of two lists.
28,multiply_numeric,transform,Element-wise multiplication of two lists.
29,longitude,transform,Returns the second tuple value in a list of LatLong tuples.
30,multiply_boolean,transform,Element-wise multiplication of two lists of boolean values.
31,cum_sum,transform,Calculates the cumulative sum.


## Deep Feature Synthesis
Deep Feature Synthesis (DFS) is the process featuretools uses to make new features. DFS stacks feature primitives to form features with a "depth" equal to the number of primitives. 
>For example, if we take the maximum value of a client's previous loans (say MAX(previous.loan_amount)), that is a "deep feature" with a depth of 1. To create a feature with a depth of two, we could stack primitives by taking the maximum value of a client's average montly payments per previous loan (such as MAX(previous(MEAN(installments.payment)))). The original paper on automated feature engineering using deep feature synthesis is worth a read.

To perform DFS in featuretools, we use the dfs function passing it 
1. an entityset, 
2. the target_entity (where we want to make the features), 
3. the agg_primitives to use, 
4. the trans_primitives to use 
5. and the max_depth of the features. 

Here we will use the default aggregation and transformation primitives, a max depth of 2, and calculate primitives for the app entity. Because this process is computationally expensive, we can run the function using ```features_only = True``` to return only a list of the features and not calculate the features themselves. This can be useful to look at the resulting features before starting an extended computation.

### DFS with Default Primitives

In [27]:
# Default primitives from featuretools
default_agg_primitives =  ["sum", "std", "max", "skew", "min", "mean", "count", "percent_true", "num_unique", "mode"]
default_trans_primitives =  ["day", "year", "month", "weekday", "haversine", "num_words", "num_characters"] # {'words':'num_words', 'characters':'num_characters'}

# DFS with specified primitives
feature_names = ft.dfs(entityset = es, target_entity = 'app',
                       trans_primitives = default_trans_primitives,
                       agg_primitives=default_agg_primitives, 
                       max_depth = 2, features_only=True)

print('%d Total Features' % len(feature_names))

2221 Total Features


If you are interested in running this call on the entire dataset and making the features, I wrote a script for that here. Unfortunately, this will not run in a Kaggle kernel due to the computational expense of the operation. Using a computer with 64GB of ram, this function call took around 24 hours (I don't think I'm technically breaking the rules of my university's high powered computing center). I have made the entire dataset available here in the file called feature_matrix.csv.

To generate a subset of the features, run the code cell below.

In [28]:
# DFS with default primitives
feature_matrix, feature_names = ft.dfs(entityset = es, target_entity = 'app',
                                       trans_primitives = default_trans_primitives,
                                       agg_primitives=default_agg_primitives, 
                                        max_depth = 2, features_only=False, verbose = True)

pd.options.display.max_columns = 1700
feature_matrix.head(10)

Built 2221 features
Elapsed: 00:18 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


AMT_ANNUITY  AMT_CREDIT  AMT_GOODS_PRICE  AMT_INCOME_TOTAL  \
SK_ID_CURR                                                               
100002          24700.5    406597.5         351000.0          202500.0   
100003          35698.5   1293502.5        1129500.0          270000.0   
100004           6750.0    135000.0         135000.0           67500.0   
100006          29686.5    312682.5         297000.0          135000.0   
100007          21865.5    513000.0         513000.0          121500.0   
100008          27517.5    490495.5         454500.0           99000.0   
100009          41301.0   1560726.0        1395000.0          171000.0   
100010          42075.0   1530000.0        1530000.0          360000.0   
100011          33826.5   1019610.0         913500.0          112500.0   
100012          20250.0    405000.0         405000.0          135000.0   

            AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_HOUR  \
SK_ID_CURR                                                          
100002                            0.0                         0.0   
100003                            0.0                         0.0   
100004                            0.0                         0.0   
100006                            NaN                         NaN   
100007                            0.0                         0.0   
100008                            0.0                         0.0   
100009                            0.0                         0.0   
100010                            0.0                         0.0   
100011                            0.0                         0.0   
100012                            NaN                         NaN   

            AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
SK_ID_CURR                                                         
100002                            0.0                        0.0   
100003                            0.0                        0.0   
100004                            0.0                        0.0   
100006                            NaN                        NaN   
100007                            0.0                        0.0   
100008                            0.0                        1.0   
100009                            1.0                        1.0   
100010                            0.0                        0.0   
100011                            0.0                        0.0   
100012                            NaN                        NaN   

            AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_YEAR  \
SK_ID_CURR                                                           
100002                             0.0                         1.0   
100003                             0.0                         0.0   
100004                             0.0                         0.0   
100006                             NaN                         NaN   
100007                             0.0                         0.0   
100008                             0.0                         1.0   
100009                             0.0                         2.0   
100010                             0.0                         0.0   
100011                             0.0                         1.0   
100012                             NaN                         NaN   

            APARTMENTS_AVG  APARTMENTS_MEDI  APARTMENTS_MODE  \
SK_ID_CURR                                                     
100002              0.0247           0.0250           0.0252   
100003              0.0959           0.0968           0.0924   
100004                 NaN              NaN              NaN   
100006                 NaN              NaN              NaN   
100007                 NaN              NaN              NaN   
100008                 NaN              NaN              NaN   
100009                 NaN              NaN              NaN   
100010                 NaN              NaN              NaN

In [32]:
feature_matrix.shape

(2002, 2221)

In [33]:
feature_names[-20:]

[<Feature: NUM_UNIQUE(credit.previous.NAME_PORTFOLIO)>,
 <Feature: NUM_UNIQUE(credit.previous.NAME_CONTRACT_TYPE)>,
 <Feature: NUM_UNIQUE(credit.previous.FLAG_LAST_APPL_PER_CONTRACT)>,
 <Feature: MODE(credit.previous.NAME_CONTRACT_STATUS)>,
 <Feature: MODE(credit.previous.NAME_TYPE_SUITE)>,
 <Feature: MODE(credit.previous.NAME_PRODUCT_TYPE)>,
 <Feature: MODE(credit.previous.NAME_CASH_LOAN_PURPOSE)>,
 <Feature: MODE(credit.previous.PRODUCT_COMBINATION)>,
 <Feature: MODE(credit.previous.NAME_SELLER_INDUSTRY)>,
 <Feature: MODE(credit.previous.NAME_YIELD_GROUP)>,
 <Feature: MODE(credit.previous.NAME_PAYMENT_TYPE)>,
 <Feature: MODE(credit.previous.CODE_REJECT_REASON)>,
 <Feature: MODE(credit.previous.NAME_CLIENT_TYPE)>,
 <Feature: MODE(credit.previous.CHANNEL_TYPE)>,
 <Feature: MODE(credit.previous.SK_ID_CURR)>,
 <Feature: MODE(credit.previous.NAME_GOODS_CATEGORY)>,
 <Feature: MODE(credit.previous.WEEKDAY_APPR_PROCESS_START)>,
 <Feature: MODE(credit.previous.NAME_PORTFOLIO)>,
 <Feature: MOD

### DFS with Selected Aggregation Primitives
With featuretools, we were able to go from 121 original features to 2221 in a few lines of code. When I did feature engineering by hand, it took about 12 hours to create a comparable size dataset. However, while we get a lot of features in featuretools, this function call is not very well-informed. We simply used the default aggregations without thinking about which ones are "important" for the problem. We end up with a lot of features, but they are probably not all relevant to the problem. Too many irrelevant features can decrease performance by drowning out the important features (related to the curse of dimensionality)

The next call we make will specify a smaller set of features. We still are not using much domain knowledge, but this feature set will be more manageable. The next step from here is improving the features we actually build and performing feature selection.
 - 여기서 tarket과 correlation이 낮은 것은 drop 하겠지? 

In [35]:
# Specify the aggregation primitives
feature_matrix_spec, feature_names_spec = ft.dfs(entityset = es, target_entity = 'app',  
                                                 agg_primitives = ['sum', 'count', 'min', 'max', 'mean', 'mode'], 
                                                 max_depth = 2, features_only = False, verbose = True)

Built 1215 features
Elapsed: 00:05 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


In [36]:
feature_matrix_spec.shape

(2002, 1215)

That "only" gives us 1215 features (and takes about 12 hours to run on the complete dataset).

In [37]:
pd.options.display.max_columns = 1215
feature_matrix_spec.head(10)

AMT_ANNUITY  AMT_CREDIT  AMT_GOODS_PRICE  AMT_INCOME_TOTAL  \
SK_ID_CURR                                                               
100002          24700.5    406597.5         351000.0          202500.0   
100003          35698.5   1293502.5        1129500.0          270000.0   
100004           6750.0    135000.0         135000.0           67500.0   
100006          29686.5    312682.5         297000.0          135000.0   
100007          21865.5    513000.0         513000.0          121500.0   
100008          27517.5    490495.5         454500.0           99000.0   
100009          41301.0   1560726.0        1395000.0          171000.0   
100010          42075.0   1530000.0        1530000.0          360000.0   
100011          33826.5   1019610.0         913500.0          112500.0   
100012          20250.0    405000.0         405000.0          135000.0   

            AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_HOUR  \
SK_ID_CURR                                                          
100002                            0.0                         0.0   
100003                            0.0                         0.0   
100004                            0.0                         0.0   
100006                            NaN                         NaN   
100007                            0.0                         0.0   
100008                            0.0                         0.0   
100009                            0.0                         0.0   
100010                            0.0                         0.0   
100011                            0.0                         0.0   
100012                            NaN                         NaN   

            AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
SK_ID_CURR                                                         
100002                            0.0                        0.0   
100003                            0.0                        0.0   
100004                            0.0                        0.0   
100006                            NaN                        NaN   
100007                            0.0                        0.0   
100008                            0.0                        1.0   
100009                            1.0                        1.0   
100010                            0.0                        0.0   
100011                            0.0                        0.0   
100012                            NaN                        NaN   

            AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_YEAR  \
SK_ID_CURR                                                           
100002                             0.0                         1.0   
100003                             0.0                         0.0   
100004                             0.0                         0.0   
100006                             NaN                         NaN   
100007                             0.0                         0.0   
100008                             0.0                         1.0   
100009                             0.0                         2.0   
100010                             0.0                         0.0   
100011                             0.0                         1.0   
100012                             NaN                         NaN   

            APARTMENTS_AVG  APARTMENTS_MEDI  APARTMENTS_MODE  \
SK_ID_CURR                                                     
100002              0.0247           0.0250           0.0252   
100003              0.0959           0.0968           0.0924   
100004                 NaN              NaN              NaN   
100006                 NaN              NaN              NaN   
100007                 NaN              NaN              NaN   
100008                 NaN              NaN              NaN   
100009                 NaN              NaN              NaN   
100010                 NaN              NaN              NaN

### Notes on Basic Implementation
These calls represent only a small fraction of the ability of featuretools. We did not specify the variable types when creating entities, did not use the relative time variables, and didn't touch on custom primitives or seed features or interesting values! Nonetheless, in this notebook, we were able to learn the basic foundations which will allow us to more effective use the tool as we learn how it works. Now, let's take a look at some of the features we have built and modeling results.

### Results
To determine whether our basic implementation of featuretools was useful, we can look at several results:

- Cross validation scores and public leaderboard scores using several different sets of features.
- Correlations: both between the features and the ```TARGET```, and between features themselves
- Feature importances: determined by a gradient boosting machine model